In [25]:
from langchain_ollama import OllamaEmbeddings,ChatOllama
embedding_model=OllamaEmbeddings(model="nomic-embed-text:latest")
llm=ChatOllama(model="phi4:latest")

In [26]:
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [27]:
import bs4
loader=WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(
        class_=("post-content","post-title","post-header")
    )
    )
)
docs=loader.load()

In [28]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vector_store=Chroma.from_documents(documents=splits,embedding=embedding_model)

In [29]:
retriever=vector_store.as_retriever()

In [30]:
system_prompt=(
    "you are an assistant for question answering tasks."
    "using the following pieces of retrieved context to answer"
    "the question. If you don't know the answer,say that you"
    "don't know. Use three sentences maximum and keep the answer concise.\n\n"
    "{context}"
)
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [31]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)

In [32]:
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [33]:
response=rag_chain.invoke({"input":"What is self-reflection"})

In [34]:
response["answer"]

'Self-reflection is a process allowing autonomous agents to iteratively improve by refining past action decisions and correcting mistakes, which is crucial in tasks where trial and error are inevitable. It involves generating reasoning traces in natural language to evaluate actions and outcomes. In the ReAct framework, this reflection guides future behavior through explicit steps like "Thought," "Action," and "Observation."'

### Create chat history

In [35]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
 

In [36]:

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [37]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

In [38]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [39]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [40]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is the self reflection?"
response=rag_chain.invoke({"input":question,"chat_history":chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response["answer"])
    ]
)

response=rag_chain.invoke({"input":"Tell me more about it","chat_history":chat_history})

print(response["answer"])

Self-reflection in autonomous agents is implemented through frameworks like ReAct (Yao et al., 2023) and Reflexion (Shinn & Labash, 2023). In ReAct, the process includes explicit steps: "Thought," where the agent considers its plan; "Action," where it performs a task-specific or language-based action; and "Observation," where it reviews the outcome. This iterative cycle helps refine decision-making.

Reflexion extends this by adding dynamic memory and self-reflection capabilities, allowing agents to compute heuristics after each action and decide whether to reset the environment for better performance. These frameworks enhance reasoning skills in knowledge-intensive and decision-making tasks by integrating reasoning with actionable steps, showing improved results over simpler methods like Act-only baselines.
